In [8]:
import torch
from PIL import Image
from efficientnet_pytorch import EfficientNet
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

model = EfficientNet.from_pretrained('efficientnet-b3')

# Freeze weights
for param in model.parameters():
    param.requires_grad = False
in_features = model._fc.in_features
model._fc = torch.nn.Linear(in_features, 10)

transform = A.Compose(
    [
        A.Resize(256, 256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

model.cpu()
model.load_state_dict(torch.load('weights.pth', map_location=torch.device('cpu')))
def inference(imagepath, model):
    image = Image.open(imagepath)
    mapper = {   
        '0': 'Shih-Tzu',
        '1': 'Rhodesian ridgeback',
        '2': 'Beagle',
        '3': 'English foxhound',
        '4': 'Australian terrier',
        '5': 'Border terrier',
        '6': 'Golden retriever',
        '7': 'Old English sheepdog',
        '8': 'Samoyed',
        '9': 'Dingo'
    }
    model.eval()
    x = np.array(image.convert('RGB'))
    x_processed = transform(image=x)['image']
    y_hat = model(x_processed.unsqueeze(0))
    preds = y_hat.argmax(dim=1).item()
    return mapper[str(preds)], torch.max(torch.softmax(y_hat, dim=1)).item()


Loaded pretrained weights for efficientnet-b3


In [9]:
inference(
    r'C:\Users\avkos\Desktop\idp_flask\uploads\Beagle_600.jpg',
    model
)

('Beagle', 0.9325783848762512)